# Problem 1: Data with R

In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R

library(tidyverse)

In [ ]:
%%R
# make sure to get your data in the same folder, next to the notebook file
# so that this will run!


dat <- read_csv('classification_data.csv')

In [ ]:
%%R

# Your data consists of an "X" column and a "label" column
# Use ggplot to plot a kernel density estimate of each label (overlayed on the same plot)
# hint: don't forget to make sure the labels are factors!

dat %>% mutate(label = parse_factor(label, NULL)) %>% ggplot(aes(X, colour = label)) + geom_density()

# Problem 2: Using Classes in Python

In [21]:
from abc import ABC, abstractmethod
from math import sqrt

def smart_zip(a,b):
    try:
        return zip(a, b)
    except TypeError:
        return zip([a], [b])

class AbstractKNeighbors(ABC):
    def __init__(self, K):
        self.K = K

    def fit(self, X, y):
        """ Train the model!

        X should be a list of data points
        y should be a list of labels
        """
        self.X = X
        self.y = y
        return self

    @abstractmethod
    def _make_prediction(self, labels):
        pass

    def predict(self, new_x):
        """Find the nearest K neighbors
        
        new_x should be a single data point
        """

        dists = [sqrt(sum(([(i-j)**2 for i,j in smart_zip(x,new_x)])))
                 for x in self.X]
        sorted_neighbors = sorted(enumerate(dists),
                                  key=lambda t: t[1])
        labels = [self.y[i] for i,_ in sorted_neighbors]
        return self._make_prediction(labels)


class KNearestNeighbors(AbstractKNeighbors):
    def _make_prediction(self, labels):
        avg = sum(labels[:self.K])/self.K
        return round(avg)
 READ
class KFurthestNeighbors(AbstractKNeighbors):
    def _make_prediction(self, labels):
        avg = sum(labels[-self.K:])/self.K
        return round(1 - avg)

In [6]:
from csv import reader

with open('classification_data.csv', 'r') as f:
    dat = list(reader(f))[1:]
    dat = [[float(x), int(label)] for x,label in dat]

## Problem 2.1: Shuffling!

In [7]:
# In your data, "X" is a data point that is nothing more than
# a single number. 
# Shuffle your data into a random order (use random.shuffle!)

import random

random.shuffle(dat)

In [8]:
# If you shuffled your data, this test should pass
# (i.e. not throw an error)

assert(sum([label for x,label in dat[:50]]) != 0)

## Problem 2.2: Splitting!

In [22]:
# Split your data, which is now a list, into 2 sublists:
# "train" and "test"
# The "train" group should have 700 elements
# The test group should have 300 element
# Each group should have the same format as the original data

train, test = dat[:700], dat[700:]

In [23]:
# Now you will need to make another split, within the groups!
# For each group ("train" and "test") split the X's from the labels.

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

## Problem 2.3: Testing models!

In [41]:
# For each model: 
# 1. Create an instance the class, with constructor parameters: K=5
# 2. Train the instance on the "train" groups X's and labels (y's)
# 3. Test how well the instance does: 
#    A. Use the trained instance to predict the label of each "X" in the "test" group
#    B. Use your "test" labels to see if the predicted label is equal the true label


def train_and_predict(model, X_train, y_train, X_test):
    model.fit(X_train, y_train)
    return [model.predict(x) for x in X_test]

def accuracy(preds, y):
    z = [p == label for p,label in zip(preds, y)]
    return sum(z) / len(z)

models = [
    KNearestNeighbors(5), 
    KFurthestNeighbors(5)
]

preds = [train_and_predict(model, X_train, y_train, X_test) 
         for model in models]

scores = [accuracy(p,y_test) for p in preds]

In [42]:
# Compare the two classes of models!
scores

[0.6666666666666666, 0.6866666666666666]